In [28]:
import numpy as np
import json
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nltk import word_tokenize
import pandas as pd
from torch.utils.data import DataLoader,Dataset
import os
from tqdm import tqdm

In [29]:
file_path = os.getcwd()
data = pd.read_csv('./dataset/data.csv') #loading the data
print(data)
with open("./word_freq.json", "r") as json_file: #loading the unigram frequencies
    word_freq = json.load(json_file)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                                              paragraph
0     Between 1995 and 2010, a study was conducted r...
1     Poverty represents a worldwide crisis. It is t...
2     The left chart shows the population change hap...
3     Human beings are facing many challenges nowada...
4     Information about the thousands of visits from...
...                                                 ...
998   Efficient Learning of Continuous-Time Hidden M...
999   Expectation Particle Belief Propagation. We pr...
1000  Latent Bayesian melding for integrating indivi...
1001  Distributionally Robust Logistic Regression. T...
1002  Variational Dropout and the Local Reparameteri...

[1003 rows x 1 columns]


In [30]:
def preprocess_text(text): #tokenisation
    text = text.lower()
    words = word_tokenize((re.sub(r'([.,!?@#:$%^&*()_+=-])',' ',text)))
    return words

data['tokens'] = data['paragraph'].apply(preprocess_text)

print(data['tokens'].head())

0    [between, 1995, and, 2010, a, study, was, cond...
1    [poverty, represents, a, worldwide, crisis, it...
2    [the, left, chart, shows, the, population, cha...
3    [human, beings, are, facing, many, challenges,...
4    [information, about, the, thousands, of, visit...
Name: tokens, dtype: object


In [31]:
sequence_length = 2  # Number of words in the input sequence
inputs = []
targets = []
for tokens in data['tokens']:
    if len(tokens) < sequence_length + 1:
        continue  # skip short sentences
    for i in range(sequence_length, len(tokens)):
        input_seq = tokens[i-sequence_length:i]
        target = tokens[i]
        inputs.append(input_seq)
        targets.append(target)

print(f'Number of sequences: {len(inputs)}')
print(f'First input sequence: {inputs[0]}')
print(f'First target: {targets[0]}')

Number of sequences: 211598
First input sequence: ['between', '1995']
First target: and


In [32]:
min_freq = 2  
vocab = [word for word, count in word_freq.items() if count >= min_freq]
vocab = ['<PAD>', '<UNK>'] + vocab  #special tokens

word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

vocab_size = len(word_to_idx)
print(f'Vocabulary size: {vocab_size}')
print(f'First 10 words in vocabulary: {vocab[:10]}')

inputs_int = [[word_to_idx.get(word,word_to_idx['<UNK>']) for word in input] for input in inputs]
targets_int = [word_to_idx.get(word,word_to_idx['<UNK>']) for word in targets]


Vocabulary size: 7314
First 10 words in vocabulary: ['<PAD>', '<UNK>', 'between', '1995', 'and', '2010', 'a', 'study', 'was', 'conducted']


In [33]:
with open("word_to_idx.json", "w") as json_file:
    json.dump(word_to_idx, json_file)

with open("idx_to_word.json", "w") as json_file:
    json.dump(idx_to_word, json_file)

In [34]:
train_inputs = np.array(inputs_int)
train_targets = np.array(targets_int)
print(f'Training samples: {train_inputs.shape[0]}')

Training samples: 211598


In [35]:
class NextWordDataset(Dataset): #Creating the dataloader for training the models
    def __init__(self, X, y):
        self.X = torch.tensor(X)
        self.y = torch.tensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = NextWordDataset(train_inputs, train_targets)
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [37]:
#We first train the LSTM model
from lstm_bilstm import LSTM_Model
embedding_dim = 256
hidden_size = 128
num_layers = 2          
lstm_dropout = 0.3

lstm_model = LSTM_Model(
    embedding_dim=embedding_dim,
    hidden_size=hidden_size,
    num_layers=num_layers,
    vocab_size=vocab_size,
    lstm_dropout=lstm_dropout,
    bidirectional=False
)
lstm_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, lstm_model.parameters()),
    lr=0.001,
)
print(lstm_model)

LSTM_Model(
  (embeddings): Embedding(7314, 256, padding_idx=0)
  (lstm): LSTM(256, 128, num_layers=2, batch_first=True, dropout=0.3)
  (fc): Linear(in_features=128, out_features=7314, bias=True)
)


In [38]:
def train_model(model,train_loader,bidirectional = False):
    if bidirectional:
        s = 'bilstm'
    else:
        s = 'lstm'    
    checkpoint_dir = file_path + '/model_checkpoints/' + s
    num_epochs = 100
    patience = 5
    best_train_loss = float('inf') 
    early_stop_counter = 0
    top_k = 5

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        correct_topk_predictions = 0
        total_predictions = 0
        
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            
            for batch_X, batch_y in tepoch:
                targets = batch_y.to(device)
                outputs = model(batch_X.to(device)).to(device)
                loss = criterion(outputs, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step() 
                epoch_loss += loss.item()

                _, topk_predictions = torch.topk(outputs, k=top_k, dim=1)
                correct_topk_predictions += (topk_predictions == targets.view(-1, 1)).sum().item()
                total_predictions += targets.size(0)
                
                topk_accuracy = correct_topk_predictions / total_predictions
                tepoch.set_postfix(loss=loss.item(), topk_accuracy=topk_accuracy)
        
        avg_train_loss = epoch_loss / len(train_loader)
        train_topk_accuracy = correct_topk_predictions / total_predictions
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Top-{top_k} Accuracy: {train_topk_accuracy:.4f}')
        
        if avg_train_loss < best_train_loss:
            best_train_loss = avg_train_loss
            early_stop_counter = 0
            
            checkpoint_path = os.path.join(checkpoint_dir, f'best_model.pth')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss
            }, checkpoint_path)
        else:
            early_stop_counter += 1
            print(f'No improvement in training loss. Early stop counter: {early_stop_counter}/{patience}')
        
        if early_stop_counter >= patience:
            print("Early stopping triggered. Stopping training.")
            break
        


In [39]:
#Training the LSTM model
train_model(model=lstm_model,train_loader=train_loader,bidirectional=False)

Epoch [1/100]: 100%|██████████| 1654/1654 [00:03<00:00, 459.62batch/s, loss=6.93, topk_accuracy=0.221]


Epoch [1/100], Training Loss: 6.4716, Top-5 Accuracy: 0.2210


Epoch [2/100]: 100%|██████████| 1654/1654 [00:03<00:00, 485.33batch/s, loss=5.74, topk_accuracy=0.282]


Epoch [2/100], Training Loss: 5.7979, Top-5 Accuracy: 0.2816


Epoch [3/100]: 100%|██████████| 1654/1654 [00:03<00:00, 481.99batch/s, loss=5.06, topk_accuracy=0.31] 


...

Epoch [98/100], Training Loss: 2.7054, Top-5 Accuracy: 0.6557


Epoch [99/100]: 100%|██████████| 1654/1654 [00:03<00:00, 475.36batch/s, loss=2.64, topk_accuracy=0.655]


Epoch [99/100], Training Loss: 2.7041, Top-5 Accuracy: 0.6555


Epoch [100/100]: 100%|██████████| 1654/1654 [00:03<00:00, 492.80batch/s, loss=2.38, topk_accuracy=0.655]


Epoch [100/100], Training Loss: 2.7040, Top-5 Accuracy: 0.6546


In [40]:
#Now we train the bidirectional LSTM model
embedding_dim = 256
hidden_size = 128
num_layers = 2         
lstm_dropout = 0.3

bilstm_model = LSTM_Model(
    embedding_dim=embedding_dim,
    hidden_size=hidden_size,
    num_layers=num_layers,
    vocab_size=vocab_size,
    lstm_dropout=lstm_dropout,
    bidirectional=True
)
bilstm_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, bilstm_model.parameters()),
    lr=0.001,
)
print(bilstm_model)

LSTM_Model(
  (embeddings): Embedding(7314, 256, padding_idx=0)
  (lstm): LSTM(256, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=256, out_features=7314, bias=True)
)


In [41]:
train_model(model=bilstm_model,train_loader=train_loader,bidirectional=True)

Epoch [1/100]: 100%|██████████| 1654/1654 [00:04<00:00, 383.44batch/s, loss=4.59, topk_accuracy=0.254]


Epoch [1/100], Training Loss: 6.1938, Top-5 Accuracy: 0.2536


Epoch [2/100]: 100%|██████████| 1654/1654 [00:04<00:00, 385.53batch/s, loss=6.79, topk_accuracy=0.319]


Epoch [2/100], Training Loss: 5.4453, Top-5 Accuracy: 0.3193


Epoch [3/100]: 100%|██████████| 1654/1654 [00:04<00:00, 397.66batch/s, loss=6.41, topk_accuracy=0.356]


...

Epoch [98/100], Training Loss: 2.2190, Top-5 Accuracy: 0.7317


Epoch [99/100]: 100%|██████████| 1654/1654 [00:04<00:00, 393.23batch/s, loss=2.7, topk_accuracy=0.732] 


Epoch [99/100], Training Loss: 2.2198, Top-5 Accuracy: 0.7317
No improvement in training loss. Early stop counter: 1/5


Epoch [100/100]: 100%|██████████| 1654/1654 [00:04<00:00, 387.29batch/s, loss=2.09, topk_accuracy=0.734]

Epoch [100/100], Training Loss: 2.2201, Top-5 Accuracy: 0.7336
No improvement in training loss. Early stop counter: 2/5


In [42]:
lstm_check_point_path = './model_checkpoints/lstm/best_model.pth'
bilstm_check_point_path = './model_checkpoints/bilstm/best_model.pth'
lstm_check_point = torch.load(lstm_check_point_path,weights_only = True)
bilstm_check_point = torch.load(bilstm_check_point_path,weights_only = True)

lstm_model.load_state_dict(lstm_check_point['model_state_dict'])
bilstm_model.load_state_dict(bilstm_check_point['model_state_dict'])

<All keys matched successfully>

In [43]:
# Autoregressively keep on predicting next word
def predict_next_words(model, input_text,n,show_top_5=False):
    text = input_text.lower()
    model.eval()
    with torch.no_grad():
        for i in range(n):
            if i == 0:
                tokens = preprocess_text(text)
                input_ids = [word_to_idx[token] for token in tokens]
            else:
                input_ids.append(best_prediction)    
            outputs = model(torch.tensor(input_ids).unsqueeze(0).to(device))
            predictions = torch.topk(outputs, dim=1,k=5)[1]
            best_prediction = predictions[0][0]        
            predicted_words = [idx_to_word.get(idx.item(), '<UNK>') for idx in predictions[0]]
            if show_top_5:
                print(f"Top five predicted words - {predicted_words}")
            text = text + ' ' + predicted_words[0] 
            print(f"updated sentence --> {text}\n")   
  
    return text

LSTM next word generation

In [48]:
sample_texts = ['generative model','they are','it is believed']
for sample in sample_texts:
    print(f"Input text --> {sample}")
    predict_next_words(model=lstm_model, input_text=sample, n = 7, show_top_5=True)


Input text --> generative model
Top five predicted words - ['classifier', 'for', 'belonging', 'can', 'of']
updated sentence --> generative model classifier

Top five predicted words - ['we', 'to', 'that', 'designed', 'classifier']
updated sentence --> generative model classifier we

Top five predicted words - ['study', 'show', 'present', 'demonstrate', 'propose']
updated sentence --> generative model classifier we study

Top five predicted words - ['the', 'here', 'and', 'this', 'natural']
updated sentence --> generative model classifier we study the

Top five predicted words - ['physical', 'algorithm', 'development', 'joint', 'performance']
updated sentence --> generative model classifier we study the physical

Top five predicted words - ['outlook', 'steam', 'properties', '<UNK>', 'model']
updated sentence --> generative model classifier we study the physical outlook

Top five predicted words - ['in', 'that', 'a', 'the', 'our']
updated sentence --> generative model classifier we study 

BiLSTM next word generation

In [49]:
for sample in sample_texts:
    print(f"Input text --> {sample}")
    predict_next_words(model=bilstm_model, input_text=sample, n = 7, show_top_5=True)

Input text --> generative model
Top five predicted words - ['for', 'classifier', 'belonging', 'of', 'specifically']
updated sentence --> generative model for

Top five predicted words - ['each', '3d', 'the', 'sequential', 'random']
updated sentence --> generative model for each

Top five predicted words - ['long', 'operating', 'model', 'classification', 'unit']
updated sentence --> generative model for each long

Top five predicted words - ['learning', 'inputs', 'term', 'model', 'sequential']
updated sentence --> generative model for each long learning

Top five predicted words - ['methods', 'we', 'of', 'from', 'bayesian']
updated sentence --> generative model for each long learning methods

Top five predicted words - ['for', 'of', 'in', 'we', 'and']
updated sentence --> generative model for each long learning methods for

Top five predicted words - ['gaussian', 'convolutional', 'a', 'sparse', 'multi']
updated sentence --> generative model for each long learning methods for gaussian

I